# Extracción de Entidades Nombradas con CRF

En esta práctica, implementaremos un sistema de reconocimiento de entidades nombradas utilizando Conditional Random Fields (CRF). Utilizaremos el corpus CONLL2002 que contiene textos en español y holandés, con anotaciones para cuatro tipos de entidades: LOC (ubicaciones), MISC (miscelánea), ORG (organizaciones), y PER (personas).

Seguiremos un enfoque incremental:
1. Exploraremos el dataset y su estructura
2. Definiremos diferentes conjuntos de características (features)
3. Entrenaremos modelos CRF con diferentes configuraciones
4. Evaluaremos el rendimiento utilizando diversas métricas
5. Optimizaremos nuestro modelo mediante experimentación

## 1. Configuración Inicial

Primero, importaremos las bibliotecas necesarias y cargaremos los datos básicos.

In [1]:
import re
import nltk
import pandas as pd
from nltk.corpus import conll2002
from nltk.tag import CRFTagger
from sklearn.metrics import classification_report
from typing import List, Dict, Tuple
from sklearn.metrics import balanced_accuracy_score

# Descargar los datos necesarios
nltk.download('conll2002')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

# Cargar gazetteers (listas de entidades conocidas)
with open("locations.txt", encoding="utf-8") as f:
    locations_set = set(line.strip().lower() for line in f if line.strip())
    print(f"Cargadas {len(locations_set)} ubicaciones en el gazetteer")

with open("person_names.txt", encoding="utf-8") as f:
    person_names_set = set(line.strip().lower() for line in f if line.strip())
    print(f"Cargados {len(person_names_set)} nombres en el gazetteer")

Cargadas 106150 ubicaciones en el gazetteer
Cargados 123466 nombres en el gazetteer


[nltk_data] Downloading package conll2002 to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package conll2002 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\11ser\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Exploración de los datos

Veamos cómo están estructurados los datos de CONLL2002 y analicemos algunos ejemplos.

In [2]:
# Cargar conjuntos de datos en español
esp_train = list(conll2002.iob_sents('esp.train'))
esp_dev = list(conll2002.iob_sents('esp.testa'))
esp_test = list(conll2002.iob_sents('esp.testb'))

# Cargar conjuntos de datos en holandés
ned_train = list(conll2002.iob_sents('ned.train'))
ned_dev = list(conll2002.iob_sents('ned.testa'))
ned_test = list(conll2002.iob_sents('ned.testb'))

# Mostrar información sobre los conjuntos de datos
print(f"Español - Train: {len(esp_train)} oraciones, Dev: {len(esp_dev)} oraciones, Test: {len(esp_test)} oraciones")
print(f"Holandés - Train: {len(ned_train)} oraciones, Dev: {len(ned_dev)} oraciones, Test: {len(ned_test)} oraciones")

# Ver un ejemplo de oración
print("\nEjemplo de oración en español:")
print(esp_train[0])

# Analizar la distribución de etiquetas
def count_tags(dataset):
    tag_counts = {}
    for sent in dataset:
        for _, _, tag in sent:
            tag_counts[tag] = tag_counts.get(tag, 0) + 1
    return tag_counts

esp_tags = count_tags(esp_train)
print("\nDistribución de etiquetas en español (train):")
for tag, count in sorted(esp_tags.items(), key=lambda x: x[1], reverse=True):
    print(f"{tag}: {count}")

Español - Train: 8323 oraciones, Dev: 1915 oraciones, Test: 1517 oraciones
Holandés - Train: 15806 oraciones, Dev: 2895 oraciones, Test: 5195 oraciones

Ejemplo de oración en español:
[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]

Distribución de etiquetas en español (train):
O: 231920
B-ORG: 7390
I-ORG: 4992
B-LOC: 4913
B-PER: 4321
I-PER: 3903
I-MISC: 3212
B-MISC: 2173
I-LOC: 1891


## 2. Clase para Manejo de Datos

Ahora implementaremos la clase `NERDataProcessor` que facilitará la carga y transformación de los datos.

In [3]:
class NERDataProcessor:
    def __init__(self, language: str = "spanish"):
        """
        Inicializa el procesador de datos NER.
        
        Args:
            language: Idioma de los datos ('spanish' o 'dutch')
        """
        self.language = language
        
    def load_data(self):
        """
        Carga los conjuntos de datos train, dev y test.
        
        Returns:
            Tupla de (train, dev, test)
        """
        if self.language == "spanish":
            return (
                conll2002.iob_sents('esp.train'),
                conll2002.iob_sents('esp.testa'),
                conll2002.iob_sents('esp.testb')
            )
        return (
            conll2002.iob_sents('ned.train'),
            conll2002.iob_sents('ned.testa'),
            conll2002.iob_sents('ned.testb')
        )
    
    def convert_to_features(self, data):
        """
        Convierte los datos a formato de características (solo palabra y POS).
        
        Args:
            data: Datos en formato [(palabra, pos, etiqueta), ...]
            
        Returns:
            Lista de oraciones con tokens [(palabra, pos), ...]
        """
        return [[(word, pos) for word, pos, _ in sent] for sent in data]

    def get_labels(self, data):
        """
        Extrae las etiquetas de los datos.
        
        Args:
            data: Datos en formato [(palabra, pos, etiqueta), ...]
            
        Returns:
            Lista de oraciones con etiquetas [etiqueta, ...]
        """
        return [[tag for _, _, tag in sent] for sent in data]

### Demostración del procesador de datos

Veamos cómo funciona nuestro `NERDataProcessor`:

In [4]:
# Instanciar el procesador para español
processor_es = NERDataProcessor("spanish")

# Cargar los datos
train, dev, test = processor_es.load_data()

# Convertir a features y etiquetas
X_train = processor_es.convert_to_features(train)
y_train = processor_es.get_labels(train)

print(f"Número de oraciones en train: {len(X_train)}")
print(f"Ejemplo de features para una oración:")
print(X_train[0][:5])  # Primeros 5 tokens de la primera oración
print(f"Ejemplo de etiquetas para la misma oración:")
print(y_train[0][:5])  # Etiquetas correspondientes

Número de oraciones en train: 8323
Ejemplo de features para una oración:
[('Melbourne', 'NP'), ('(', 'Fpa'), ('Australia', 'NP'), (')', 'Fpt'), (',', 'Fc')]
Ejemplo de etiquetas para la misma oración:
['B-LOC', 'O', 'B-LOC', 'O', 'O']


## 3. Generador de Características

Las características son cruciales para el rendimiento de un modelo CRF. Implementaremos un generador de características flexible que nos permitirá experimentar con diferentes combinaciones.

In [5]:
class CRFFeatureGenerator:
    def __init__(self, feature_config: Dict):
        """
        Inicializa el generador de características.
        
        Args:
            feature_config: Diccionario de configuración con banderas para activar/desactivar grupos de características
        """
        self.config = feature_config
        self.lemmatizer = nltk.WordNetLemmatizer()
        
    def get_features(self, tokens: List[Tuple[str, str]], index: int) -> List[str]:
        """
        Genera características para un token en una posición específica.
        
        Args:
            tokens: Lista de tokens de la oración [(palabra, pos), ...]
            index: Índice del token actual
            
        Returns:
            Lista de características para el token
        """
        word, pos = tokens[index]
        features = ["bias"]  # Característica base siempre presente
        
        # --- Características de la palabra ---
        if self.config.get("word_form", True):
            features.append(f"word={word}")
            features.append(f"word.lower={word.lower()}")

        # --- Características de POS y lematización ---
        if self.config.get("pos", True):
            features.append(f"pos={pos}")
            features.append(f"lemma={self.lemmatizer.lemmatize(word.lower())}")

        # --- Características morfológicas ---
        if self.config.get("morphology", True):
            features.append(f"is_title={word.istitle()}")
            features.append(f"is_upper={word.isupper()}")
            features.append(f"is_digit={word.isdigit()}")
            features.append(f"has_digit={any(c.isdigit() for c in word)}")
            features.append(f"has_symbol={not word.isalnum()}")

        # --- Prefijos y sufijos ---
        if self.config.get("prefix_suffix", True):
            if len(word) >= 3:
                features.append(f"prefix3={word[:3]}")
                features.append(f"suffix3={word[-3:]}")
            if len(word) >= 2:
                features.append(f"prefix2={word[:2]}")
                features.append(f"suffix2={word[-2:]}")

        # --- Longitud de la palabra ---
        if self.config.get("length", True):
            features.append(f"length={len(word)}")

        # --- Posición en la oración ---
        if self.config.get("position", True):
            features.append(f"position={index}")
            features.append(f"is_first={index == 0}")
            features.append(f"is_last={index == len(tokens)-1}")

        # --- Contexto circundante ---
        if self.config.get("context", True):
            # Palabra anterior
            if index > 0:
                prev_word, prev_pos = tokens[index-1]
                features.append(f"prev_word.lower={prev_word.lower()}")
                features.append(f"prev_word.istitle={prev_word.istitle()}")
                features.append(f"prev_word.isdigit={prev_word.isdigit()}")
                features.append(f"prev_pos={prev_pos}")
            else:
                features.append("BOS")  # Beginning of sentence
                
            # Palabra siguiente
            if index < len(tokens)-1:
                next_word, next_pos = tokens[index+1]
                features.append(f"next_word.lower={next_word.lower()}")
                features.append(f"next_word.istitle={next_word.istitle()}")
                features.append(f"next_word.isdigit={next_word.isdigit()}")
                features.append(f"next_pos={next_pos}")
            else:
                features.append("EOS")  # End of sentence

        # --- Características de gazetteers ---
        if self.config.get("gazetteers", True):
            features.append(f"in_location_gazetteer={word.lower() in locations_set}")
            features.append(f"in_person_gazetteer={word.lower() in person_names_set}")

        return features

### Demostración del generador de características

Veamos las características que genera nuestro `CRFFeatureGenerator` para algunos tokens:

In [19]:
# Configuración completa de características
full_config = {
    "word_form": True,
    "pos": True,
    "morphology": True,
    "prefix_suffix": True,
    "length": True,
    "position": True,
    "context": True,
    "gazetteers": True,
}

# Crear generadores de características
full_feature_gen = CRFFeatureGenerator(full_config)
# Ejemplo de oración
example_tokens = X_train[0]
example_word_index = 0  # Índice de ejemplo

print("Palabra de ejemplo:", example_tokens[example_word_index][0])

# Generar características con ambas configuraciones
full_features = full_feature_gen.get_features(example_tokens, example_word_index)

print("\nCaracterísticas completas:")
for feat in full_features[:15]:
    print(f"  - {feat}")

print(f"\nTotal de características completas: {len(full_features)}")

Palabra de ejemplo: Melbourne

Características completas:
  - bias
  - word=Melbourne
  - word.lower=melbourne
  - pos=NP
  - lemma=melbourne
  - is_title=True
  - is_upper=False
  - is_digit=False
  - has_digit=False
  - has_symbol=False
  - prefix3=Mel
  - suffix3=rne
  - prefix2=Me
  - suffix2=ne
  - length=9

Total de características completas: 25


## 4. Modelo CRF y Entrenamiento

Ahora implementaremos la clase `CRFModel` que encapsula la lógica de entrenamiento y predicción con CRF.

In [43]:
class CRFModel:
    def __init__(self, feature_generator: CRFFeatureGenerator):
        """
        Inicializa el modelo CRF.
        
        Args:
            feature_generator: Generador de características a utilizar
        """
        self.ct = CRFTagger(feature_func=feature_generator.get_features)
        
    def train(self, train_sents, train_labels, model_file='model.crf'):
        """
        Entrena el modelo CRF.
        
        Args:
            train_sents: Oraciones de entrenamiento [(palabra, pos), ...]
            train_labels: Etiquetas correspondientes [etiqueta, ...]
            model_file: Nombre del archivo donde guardar el modelo
        """
        formatted_data = self._format_data(train_sents, train_labels)
        self.ct.train(formatted_data, model_file)
        print(f"Modelo entrenado y guardado como '{model_file}'")
        
    def predict(self, test_sents):
        """
        Predice etiquetas para oraciones.
        
        Args:
            test_sents: Oraciones para predecir [(palabra, pos), ...]
            
        Returns:
            Lista de oraciones con etiquetas predichas [etiqueta, ...]
        """
        tagged_sents = self.ct.tag_sents(test_sents)
        # Extraer solo las etiquetas de las tuplas (palabra, etiqueta)
        return [[tag for _, tag in sent] for sent in tagged_sents]
    
    def _format_data(self, sents, labels):
        """
        Formatea los datos para el entrenamiento.
        
        Args:
            sents: Oraciones [(palabra, pos), ...]
            labels: Etiquetas [etiqueta, ...]
            
        Returns:
            Lista de oraciones formateadas para CRFTagger [(palabra, etiqueta), ...]
        """
        return [list(zip(sent, label)) for sent, label in zip(sents, labels)]

### Entrenamiento de un modelo básico

Entrenemos un modelo CRF básico para ver cómo funciona:

In [24]:
# Configura un modelo básico (solo forma de palabra)
basic_config = {
    "word_form": True,
    "pos": True,
    "morphology": True,
    "prefix_suffix": True,
    "length": True,
    "position": True,
    "context": True,
    "gazetteers": True,
}

# Crear generador de características y modelo
feature_gen = CRFFeatureGenerator(basic_config)
model = CRFModel(feature_gen)

# Usar un subconjunto pequeño para la demostración
small_X_train = X_train[:100]  # primeras 100 oraciones
small_y_train = y_train[:100]

# Entrena el modelo
model.train(small_X_train, small_y_train)

# Predice sobre algunas oraciones
small_X_dev = X_train[100:105]  # 5 oraciones adicionales
predictions = model.predict(small_X_dev)

# Muestra las predicciones
for i, (sentence, pred_tags) in enumerate(zip(small_X_dev, predictions)):
    print(f"\nOración {i+1}:")
    for (word, _), tag in zip(sentence, pred_tags):
        print(f"{word}\t{tag}")

Modelo entrenado y guardado como 'model.crf'

Oración 1:
Imagínense	O
ustedes	O
que	O
entre	O
aquellos	O
españoles	O
,	O
que	O
fueron	O
quienes	O
llevaron	O
a	O
Europa	B-LOC
esos	O
dones	O
americanos	O
,	O
se	O
hubiera	O
impuesto	O
la	O
patriotería	O
gastronómica	O
:	O
patatas	O
y	O
tomates	O
se	O
hubieran	O
quedado	O
en	O
curiosidades	O
botánicas	O
.	O

Oración 2:
No	O
hay	O
,	O
no	O
puede	O
haber	O
,	O
una	O
cocina	O
universal	O
;	O
recuerden	O
ustedes	O
ese	O
horror	O
conocido	O
como	O
'	O
cocina	O
internacional	O
'	O
tan	O
frecuente	O
en	O
restaurantes	O
de	O
hotel	O
.	O

Oración 3:
Pero	O
tampoco	O
puede	O
haber	O
una	O
cocina	O
encerrada	O
en	O
sí	O
misma	O
.	O

Oración 4:
No	O
cabe	O
preguntarse	O
,	O
ante	O
un	O
nuevo	O
alimento	O
,	O
una	O
nueva	O
especia	O
,	O
¿	O
de	O
dónde	O
viene	O
esto	O
?	O
,	O
sin	O
investigar	O
antes	O
lo	O
verdaderamente	O
importante	O
:	O
¿	O
está	O
rico	O
?	O

Oración 5:
Pues	O
,	O
si	O
está	O
rico	O
...	O
adelante	O
,	O
viniere	O
de	O
donde	O
vinie

## 5. Funciones de Evaluación

Para evaluar el rendimiento de nuestros modelos, implementaremos funciones que calculan diversas métricas.

In [50]:
def sent_tags_to_IO(sent_tags):
    """
    Convierte etiquetas al formato IO (B-X -> I-X).
    
    Args:
        sent_tags: Lista de oraciones con etiquetas
        
    Returns:
        Lista de oraciones con etiquetas en formato IO
    """
    return [[tag.replace("B-", "I-") for tag in sent] for sent in sent_tags]

def entity_finder(sent_tags):
    """
    Encuentra entidades en las oraciones basándose en etiquetas.
    
    Args:
        sent_tags: Lista de oraciones con etiquetas
        
    Returns:
        Lista de entidades encontradas por oración [(tipo, (inicio, fin)), ...]
    """
    entities = []
    for sent in sent_tags:
        sent_entities = []  # Lista para la oración actual
        entities.append(sent_entities)
        
        current_entity = None
        start_idx = None
        entity_type = None
        
        for i, tag in enumerate(sent):
            if tag.startswith("I-"):
                if current_entity is None:  # Nueva entidad
                    current_entity = tag[2:]
                    start_idx = i
                    entity_type = tag[2:]
                elif tag[2:] != entity_type:  # Cambio de tipo
                    if current_entity:
                        sent_entities.append((entity_type, (start_idx, i-1)))
                    current_entity = tag[2:]
                    start_idx = i
                    entity_type = tag[2:]
            else:  # "O" u otra etiqueta no-entidad
                if current_entity is not None:  # Finalizar entidad
                    sent_entities.append((entity_type, (start_idx, i-1)))
                    current_entity = None
                    start_idx = None
                    entity_type = None
        
        # Si hay una entidad al final de la oración
        if current_entity is not None:
            sent_entities.append((entity_type, (start_idx, len(sent)-1)))
    
    return entities

def evaluate_model(y_true, y_pred, errors=False):
    """
    Evalúa el rendimiento del modelo usando múltiples métricas.
    
    Args:
        y_true: Etiquetas reales
        y_pred: Etiquetas predichas
        errors: Si es True, devuelve también lista de errores
        
    Returns:
        Diccionario con métricas de rendimiento
    """
    info = {'Balanced accuracy': 0.0, 'F1 Score': 0.0, 'Precision': 0.0, 'Recall': 0.0}
    
    # Convertir a formato IO para análisis consistente
    y_true_io = sent_tags_to_IO(y_true)
    y_pred_io = sent_tags_to_IO(y_pred)

    # Calcular balanced accuracy usando solo la primera letra de la etiqueta (I/O)
    def join_sent_tags(sent_tags):
        return [tag for sent in sent_tags for tag in sent]

    info['Balanced accuracy'] = balanced_accuracy_score(join_sent_tags(y_true_io), join_sent_tags(y_pred_io))

    # Encontrar entidades
    true_entities = entity_finder(y_true_io)
    pred_entities = entity_finder(y_pred_io)

    # Contar entidades reales y correctas por tipo
    counts = {'LOC': 0, 'MISC': 0, 'ORG': 0, 'PER': 0}
    correct_counts = {'LOC': 0, 'MISC': 0, 'ORG': 0, 'PER': 0}
    invented = 0

    for i, sent in enumerate(true_entities):
        sent_true = set(sent)
        sent_pred = set(pred_entities[i])
        
        # Contar entidades reales por tipo
        for ent in sent:
            counts[ent[0]] += 1
            
        # Contar entidades correctamente predichas
        for ent in sent_pred & sent_true:
            correct_counts[ent[0]] += 1
            
        # Contar entidades inventadas (falsos positivos)
        invented += len(sent_pred - sent_true)

    # Calcular precisión por tipo de entidad
    for ent_type in counts:
        total = counts[ent_type]
        correct = correct_counts[ent_type]
        info[f'{ent_type} correct'] = correct / total if total > 0 else 0.0

    # Calcular métricas globales
    total_entities = sum(counts.values())
    true_positives = sum(correct_counts.values())
    false_positives = invented
    false_negatives = total_entities - true_positives
    
    # Precision, Recall y F1
    info['Precision'] = true_positives / (true_positives + false_positives) if (true_positives + false_positives) else 0
    info['Recall'] = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) else 0
    
    if info['Precision'] + info['Recall'] > 0:
        info['F1 Score'] = 2 * (info['Precision'] * info['Recall']) / (info['Precision'] + info['Recall'])
    else:
        info['F1 Score'] = 0

    # Devolver lista de errores si se solicita
    if errors:
        error_list = []
        for i in range(len(true_entities)):
            error_list.extend([
                (i, ent) for ent in pred_entities[i] if ent not in true_entities[i]
            ])
        return info, error_list

    return info

### Demostración de evaluación

Veamos cómo funcionan nuestras funciones de evaluación con un pequeño ejemplo:

In [51]:
# Obtener algunas etiquetas reales
# Usar un ejemplo más grande: 200 oraciones
true_tags = y_train[100:300]
# Volver a predecir para el mismo rango
pred_tags = model.predict(X_train[100:300])

# Evaluación
eval_results = evaluate_model(true_tags, pred_tags)
display(pd.DataFrame([eval_results]).T.rename(columns={0: "Resultats de l'avaluació"}))

,Resultats de l'avaluació
Balanced accuracy,0.516775
F1 Score,0.375622
Precision,0.457576
Recall,0.318565
LOC correct,0.466019
MISC correct,0.225806
ORG correct,0.219124
PER correct,0.460674


## 6. Pipeline de Experimentación

Ahora crearemos una función que encapsule todo el proceso de experimentación: entrenamiento, predicción y evaluación.

In [44]:
def run_experiment(config: Dict, language: str = "spanish", sample_size=None):
    """
    Ejecuta un experimento completo: entrenamiento, predicción y evaluación.
    
    Args:
        config: Configuración de características
        language: Idioma ('spanish' o 'dutch')
        sample_size: Número de oraciones a usar (None para usar todas)
        
    Returns:
        Resultados de evaluación
    """
    processor = NERDataProcessor(language)
    train, dev, test = processor.load_data()
    
    # Limitar el tamaño de la muestra si es necesario
    if sample_size:
        train = list(train)[:sample_size]
        dev = list(dev)[:sample_size//10]
    
    # Convertir datos
    X_train = processor.convert_to_features(train)
    y_train = processor.get_labels(train)
    X_dev = processor.convert_to_features(dev)
    y_dev = processor.get_labels(dev)
    
    # Configurar modelo
    feature_gen = CRFFeatureGenerator(config)
    model = CRFModel(feature_gen)

    config_str = "_".join([k for k, v in config.items() if v])
    model_file = f"model_{config_str}.crf"
    
    # Entrenar y predecir
    model.train(X_train, y_train, model_file=model_file)
    y_pred = model.predict(X_dev)
    
    # Evaluar
    results = evaluate_model(y_dev, y_pred)
    
    return results

### Experimento base

Ejecutemos un experimento base con características mínimas para establecer un punto de referencia:

In [52]:
# Configuración básica
basic_config = {
    "word_form": True,
    "morphology": True,
    "position": True,
    "pos": False,
    "prefix_suffix": False,
    "length": False,
    "context": False,
    "gazetteers": False,
}

# Ejecutar experimento con muestra pequeña para demostración
baseline_results = run_experiment(basic_config, sample_size=1000)

print("Resultados del experimento base:")
pd.DataFrame([baseline_results]).T

Modelo entrenado y guardado como 'model_word_form_morphology_position.crf'
Resultados del experimento base:


,0
Balanced accuracy,0.554371
F1 Score,0.464832
Precision,0.484076
Recall,0.447059
LOC correct,0.462963
MISC correct,0.100000
ORG correct,0.540984
PER correct,0.457143


## 7. Experimentación con Diferentes Características

Ahora exploraremos cómo afectan diferentes características al rendimiento del modelo.

In [53]:
import itertools
from time import time

def feature_ablation_study(language="spanish", sample_size=1000):
    """
    Realiza un estudio de ablación de características.
    
    Args:
        language: Idioma a utilizar
        sample_size: Tamaño de la muestra
        
    Returns:
        DataFrame con resultados
    """
    # Características a probar
    feature_groups = [
        "pos",
        "context",
        "prefix_suffix",
        "length",
        "gazetteers",
    ]
    
    results = []
    
    # Configuración base siempre presente
    base_config = {
        "word_form": True,  # Siempre incluimos la forma de palabra
        "position": True,
        "morphology": True,
    }
    
    # Inicializar todas las características adicionales como False
    for feature in feature_groups:
        base_config[feature] = False
    
    # 1. Probar la configuración base
    print("Evaluando configuración base...")
    start_time = time()
    result = run_experiment(base_config.copy(), language=language, sample_size=sample_size)
    end_time = time()
    
    result_with_config = {
        "Configuración": "Base",
        "Tiempo (s)": end_time - start_time
    }
    result_with_config.update(result)
    results.append(result_with_config)
    
    # 2. Probar cada característica individual añadida a la base
    print("Probando características individuales...")
    for feature in feature_groups:
        config = base_config.copy()
        config[feature] = True
        
        print(f"Evaluando configuración con {feature}")
        start_time = time()
        result = run_experiment(config, language=language, sample_size=sample_size)
        end_time = time()
        
        result_with_config = {
            "Configuración": f"Base + {feature}",
            "Tiempo (s)": end_time - start_time
        }
        result_with_config.update(result)
        results.append(result_with_config)
    
    # 3. Probar la configuración completa
    full_config = base_config.copy()
    for feature in feature_groups:
        full_config[feature] = True
    
    print("Evaluando configuración completa...")
    start_time = time()
    result = run_experiment(full_config, language=language, sample_size=sample_size)
    end_time = time()
    
    result_with_config = {
        "Configuración": "Configuración completa",
        "Tiempo (s)": end_time - start_time
    }
    result_with_config.update(result)
    results.append(result_with_config)
    
    # 4. Ablación real: quitar una característica a la vez de la configuración completa
    print("Realizando ablación (quitando una característica a la vez)...")
    for feature in feature_groups:
        config = full_config.copy()
        config[feature] = False
        
        print(f"Evaluando configuración sin {feature}")
        start_time = time()
        result = run_experiment(config, language=language, sample_size=sample_size)
        end_time = time()
        
        result_with_config = {
            "Configuración": f"Completa - {feature}",
            "Tiempo (s)": end_time - start_time
        }
        result_with_config.update(result)
        results.append(result_with_config)
    
    # Convertir a DataFrame
    results_df = pd.DataFrame(results)
    return results_df.set_index("Configuración")

# Ejecutar estudio de ablación con muestra pequeña
ablation_results = feature_ablation_study(sample_size=1000)
ablation_results

Evaluando configuración base...
Modelo entrenado y guardado como 'model_word_form_position_morphology.crf'
Probando características individuales...
Evaluando configuración con pos
Modelo entrenado y guardado como 'model_word_form_position_morphology_pos.crf'
Evaluando configuración con context
Modelo entrenado y guardado como 'model_word_form_position_morphology_context.crf'
Evaluando configuración con prefix_suffix
Modelo entrenado y guardado como 'model_word_form_position_morphology_prefix_suffix.crf'
Evaluando configuración con length
Modelo entrenado y guardado como 'model_word_form_position_morphology_length.crf'
Evaluando configuración con gazetteers
Modelo entrenado y guardado como 'model_word_form_position_morphology_gazetteers.crf'
Evaluando configuración completa...
Modelo entrenado y guardado como 'model_word_form_position_morphology_pos_context_prefix_suffix_length_gazetteers.crf'
Realizando ablación (quitando una característica a la vez)...
Evaluando configuración sin pos


,Tiempo (s),Balanced accuracy,F1 Score,Precision,Recall,LOC correct,MISC correct,ORG correct,PER correct
Configuración,,,,,,,,,
Base,4.987299,0.554371,0.464832,0.484076,0.447059,0.462963,0.10,0.540984,0.457143
Base + pos,5.440558,0.611477,0.553191,0.572327,0.535294,0.574074,0.10,0.672131,0.485714
Base + context,6.913379,0.624429,0.538922,0.548780,0.529412,0.537037,0.10,0.639344,0.571429
Base + prefix_suffix,5.637598,0.609596,0.538226,0.560510,0.517647,0.555556,0.10,0.622951,0.514286
Base + length,5.091350,0.627444,0.513433,0.521212,0.505882,0.518519,0.15,0.606557,0.514286
Base + gazetteers,5.464594,0.594577,0.536585,0.556962,0.517647,0.481481,0.05,0.721311,0.485714
Configuración completa,8.390991,0.679353,0.634731,0.646341,0.623529,0.666667,0.05,0.803279,0.571429
Completa - pos,8.402137,0.704162,0.631268,0.633136,0.629412,0.703704,0.10,0.786885,0.542857
Completa - context,6.760878,0.669249,0.610272,0.627329,0.594118,0.685185,0.05,0.737705,0.514286


### Análisis de los resultados de ablación

Analicemos qué características individuales aportan más al rendimiento del modelo:

In [55]:
# Visualizar los resultados clave
key_metrics = ["Balanced accuracy", "F1 Score", "Precision", "Recall"]
ablation_results[key_metrics].sort_values(by="Balanced accuracy", ascending=False)

,Balanced accuracy,F1 Score,Precision,Recall
Configuración,,,,
Completa - prefix_suffix,0.723128,0.664671,0.676829,0.652941
Completa - pos,0.704162,0.631268,0.633136,0.629412
Completa - length,0.699094,0.642643,0.656442,0.629412
Completa - gazetteers,0.679585,0.612613,0.625767,0.600000
Configuración completa,0.679353,0.634731,0.646341,0.623529
Completa - context,0.669249,0.610272,0.627329,0.594118
Base + length,0.627444,0.513433,0.521212,0.505882
Base + context,0.624429,0.538922,0.548780,0.529412
Base + pos,0.611477,0.553191,0.572327,0.535294


## 8. Experimentación con Contexto

Ahora exploraremos cómo afecta la inclusión de información contextual al rendimiento del modelo.

In [56]:
def context_experiment(language="spanish", sample_size=500):
    """
    Realiza experimentos con diferentes configuraciones de contexto.
    
    Args:
        language: Idioma a utilizar
        sample_size: Tamaño de la muestra
        
    Returns:
        DataFrame con resultados
    """
    # Configuración base con mejores características
    base_config = {
        "word_form": True,
        "pos": True,
        "morphology": True,
        "prefix_suffix": True,
        "length": True,
        "position": False,
        "context": False,
        "gazetteers": False
    }
    
    configs = [
        {"name": "Sin contexto", "config": base_config.copy()},
        {"name": "Con contexto", "config": {**base_config, "context": True}},
        {"name": "Con contexto + posición", "config": {**base_config, "context": True, "position": True}},
        {"name": "Completo (con gazetteers)", "config": {**base_config, "context": True, "position": True, "gazetteers": True}}
    ]
    
    results = []
    
    for cfg in configs:
        print(f"Evaluando: {cfg['name']}")
        start_time = time()
        result = run_experiment(cfg["config"], language=language, sample_size=sample_size)
        end_time = time()
        
        result_with_config = {
            "Configuración": cfg["name"],
            "Tiempo (s)": end_time - start_time
        }
        result_with_config.update(result)
        
        results.append(result_with_config)
    
    # Convertir a DataFrame
    results_df = pd.DataFrame(results)
    return results_df.set_index("Configuración")

# Ejecutar experimento de contexto
context_results = context_experiment(sample_size=1000)
context_results[["Balanced accuracy", "F1 Score", "Precision", "Recall"]]

Evaluando: Sin contexto
Modelo entrenado y guardado como 'model_word_form_pos_morphology_prefix_suffix_length.crf'
Evaluando: Con contexto
Modelo entrenado y guardado como 'model_word_form_pos_morphology_prefix_suffix_length_context.crf'
Evaluando: Con contexto + posición
Modelo entrenado y guardado como 'model_word_form_pos_morphology_prefix_suffix_length_position_context.crf'
Evaluando: Completo (con gazetteers)
Modelo entrenado y guardado como 'model_word_form_pos_morphology_prefix_suffix_length_position_context_gazetteers.crf'


,Balanced accuracy,F1 Score,Precision,Recall
Configuración,,,,
Sin contexto,0.616659,0.549849,0.565217,0.535294
Con contexto,0.661735,0.602985,0.612121,0.594118
Con contexto + posición,0.679585,0.612613,0.625767,0.600000
Completo (con gazetteers),0.679353,0.634731,0.646341,0.623529


## 9. Comparación entre Idiomas

Comparemos el rendimiento de nuestro mejor modelo en español y holandés.

In [57]:
def language_comparison(sample_size=500):
    """
    Compara el rendimiento del modelo en español y holandés.
    
    Args:
        sample_size: Tamaño de la muestra
        
    Returns:
        DataFrame con resultados
    """
    # Mejor configuración encontrada
    best_config = {
        "word_form": True,
        "pos": True,
        "morphology": True,
        "prefix_suffix": True,
        "length": True,
        "position": True,
        "context": True,
        "gazetteers": True
    }
    
    results = []
    
    # Español
    print("Evaluando modelo en español...")
    start_time = time()
    es_result = run_experiment(best_config, language="spanish", sample_size=sample_size)
    end_time = time()
    
    es_result_with_meta = {
        "Idioma": "Español",
        "Tiempo (s)": end_time - start_time
    }
    es_result_with_meta.update(es_result)
    results.append(es_result_with_meta)
    
    # Holandés
    print("Evaluando modelo en holandés...")
    start_time = time()
    nl_result = run_experiment(best_config, language="dutch", sample_size=sample_size)
    end_time = time()
    
    nl_result_with_meta = {
        "Idioma": "Holandés",
        "Tiempo (s)": end_time - start_time
    }
    nl_result_with_meta.update(nl_result)
    results.append(nl_result_with_meta)
    
    # Convertir a DataFrame
    results_df = pd.DataFrame(results)
    return results_df.set_index("Idioma")

# Comparar idiomas
language_results = language_comparison(sample_size=1000)
language_results

Evaluando modelo en español...
Modelo entrenado y guardado como 'model_word_form_pos_morphology_prefix_suffix_length_position_context_gazetteers.crf'
Evaluando modelo en holandés...
Modelo entrenado y guardado como 'model_word_form_pos_morphology_prefix_suffix_length_position_context_gazetteers.crf'


,Tiempo (s),Balanced accuracy,F1 Score,Precision,Recall,LOC correct,MISC correct,ORG correct,PER correct
Idioma,,,,,,,,,
Español,8.752378,0.679353,0.634731,0.646341,0.623529,0.666667,0.050000,0.803279,0.571429
Holandés,2.508224,0.688394,0.608696,0.653333,0.569767,0.615385,0.541667,0.478261,0.692308


## 10. Experimento con Datos Completos

Finalmente, entrenemos y evaluemos nuestro mejor modelo usando el conjunto completo de datos.

In [65]:
def full_experiment(language="spanish"):
    """
    Realiza un experimento completo con todos los datos.
    
    Args:
        language: Idioma a utilizar
        
    Returns:
        Resultados de evaluación
    """
    best_config = {
        "word_form": True,
        "pos": True,
        "morphology": True,
        "prefix_suffix": True,
        "length": True,
        "position": True,
        "context": True,
        "gazetteers": True
    }
    
    print(f"Entrenando modelo completo en {language}...")
    
    start_time = time()
    results = run_experiment(best_config, language=language)
    end_time = time()
    
    print(f"Tiempo total: {end_time - start_time:.2f} segundos")
    
    return results

# Ejecutar experimento completo (comentado por tiempo de ejecución)
full_results = full_experiment()
df_full = pd.DataFrame([full_results])
display(df_full.sort_values(by="F1 Score", ascending=False).T)

Entrenando modelo completo en spanish...
Modelo entrenado y guardado como 'model_word_form_pos_morphology_prefix_suffix_length_position_context_gazetteers.crf'
Tiempo total: 159.84 segundos


,0
Balanced accuracy,0.786713
F1 Score,0.738690
Precision,0.752882
Recall,0.725023
LOC correct,0.794450
MISC correct,0.418919
ORG correct,0.706335
PER correct,0.807061


## 11. Análisis de Errores

Analicemos algunos de los errores típicos que comete nuestro modelo para entender sus limitaciones.

In [66]:
def error_analysis(language="spanish", sample_size=500):
    """
    Analiza errores típicos del modelo.
    
    Args:
        language: Idioma a utilizar
        sample_size: Tamaño de la muestra
    """
    processor = NERDataProcessor(language)
    train, dev, test = processor.load_data()
    
    # Limitar el tamaño para análisis
    dev = list(dev)[:sample_size]
    
    # Convertir datos
    X_dev = processor.convert_to_features(dev)
    y_dev = processor.get_labels(dev)
    
    # Mejor configuración
    best_config = {
        "word_form": True,
        "pos": True,
        "morphology": True,
        "prefix_suffix": True,
        "length": True,
        "position": True,
        "context": True,
        "gazetteers": True
    }
    
    # Configurar y entrenar modelo
    feature_gen = CRFFeatureGenerator(best_config)
    model = CRFModel(feature_gen)
    
    # Cargar modelo pre-entrenado o entrenar con muestra pequeña
    try:
        print("Intentando cargar modelo pre-entrenado...")
        model.ct.set_model_file('model.crf')
    except:
        print("Entrenando nuevo modelo...")
        X_train = processor.convert_to_features(list(train)[:1000])
        y_train = processor.get_labels(list(train)[:1000])
        model.train(X_train, y_train)
    
    # Predecir
    y_pred = model.predict(X_dev)
    
    # Obtener métricas y errores
    results, errors = evaluate_model(y_dev, y_pred, errors=True)
    
    print(f"Rendimiento global (F1): {results['F1 Score']:.4f}")
    print(f"Entidades con mayor dificultad: {min(results.items(), key=lambda x: x[1] if x[0].endswith('correct') else 1.0)}")
    
    # Analizar algunos errores específicos
    print("\nAnálisis de errores comunes:")
    
    if not errors:
        print("No se encontraron errores en la muestra analizada.")
        return
    
    # Mostrar 5 errores aleatorios
    import random
    random.seed(42)
    sample_errors = random.sample(errors, min(5, len(errors)))
    
    for i, (sent_idx, entity) in enumerate(sample_errors):
        print(f"\nError {i+1}:")
        print(f"Entidad incorrectamente predicha: {entity}")
        
        # Reconstruir la oración original
        original_sent = dev[sent_idx]
        words = [word for word, _, _ in original_sent]
        true_tags = [tag for _, _, tag in original_sent]
        pred_tags = y_pred[sent_idx]
        
        # Mostrar contexto de error
        start_idx = max(0, entity[1][0] - 2)
        end_idx = min(len(words), entity[1][1] + 3)
        
        print("Contexto:")
        for j in range(start_idx, end_idx):
            prefix = "→ " if (j >= entity[1][0] and j <= entity[1][1]) else "  "
            print(f"{prefix}{words[j]} (Real: {true_tags[j]}, Pred: {pred_tags[j]})")

# Ejecutar análisis de errores
error_analysis(sample_size=200)

Intentando cargar modelo pre-entrenado...
Rendimiento global (F1): 0.6436
Entidades con mayor dificultad: ('MISC correct', 0.10256410256410256)

Análisis de errores comunes:

Error 1:
Entidad incorrectamente predicha: ('MISC', (11, 15))
Contexto:
  de (Real: I-MISC, Pred: O)
  la (Real: I-MISC, Pred: O)
→ Feria (Real: I-MISC, Pred: B-MISC)
→ de (Real: I-MISC, Pred: I-MISC)
→ Alfarería. (Real: I-MISC, Pred: I-MISC)
→ el (Real: I-MISC, Pred: I-MISC)
→ Barro (Real: I-MISC, Pred: I-MISC)
  . (Real: O, Pred: O)

Error 2:
Entidad incorrectamente predicha: ('LOC', (36, 37))
Contexto:
  Telecom (Real: I-ORG, Pred: I-ORG)
  en (Real: O, Pred: O)
→ Telesp (Real: B-ORG, Pred: B-LOC)
→ Celular (Real: I-ORG, Pred: I-LOC)
  , (Real: O, Pred: O)
  la (Real: O, Pred: O)

Error 3:
Entidad incorrectamente predicha: ('PER', (23, 24))
Contexto:
  Brasil (Real: I-MISC, Pred: I-ORG)
  , (Real: O, Pred: O)
→ Joao (Real: B-PER, Pred: B-PER)
→ Pimienta (Real: I-PER, Pred: I-PER)
  da (Real: I-PER, Pred: O)
  V

## 12. Conclusiones y Trabajo Futuro

En esta práctica, hemos implementado y evaluado un sistema de reconocimiento de entidades nombradas utilizando Conditional Random Fields. Nuestros experimentos han demostrado:

1. **Importancia de las características**: Las características morfológicas, prefijos/sufijos y POS han demostrado ser muy importantes para el rendimiento del modelo.

2. **Contexto**: La información contextual (palabras anteriores y siguientes) mejora significativamente la precisión de las predicciones.

3. **Gazetteers**: El uso de listas de entidades conocidas ayuda, especialmente para tipos específicos como personas y ubicaciones.

4. **Diferencias entre idiomas**: Hemos observado patrones diferentes de rendimiento entre español y holandés, lo que sugiere la necesidad de adaptar las características al idioma.

**Trabajo futuro**:

- Explorar modelos más avanzados como BiLSTM-CRF o transformers
- Expandir las listas de gazetteers
- Implementar técnicas de validación cruzada
- Probar diferentes esquemas de codificación (BIO, BIOES)
- Optimizar hiperparámetros del CRF

In [67]:
# Resumen de resultados
print("Resumen de experimentos:")

# Intentar recuperar resultados de experimentos anteriores si están disponibles
try:
    results_summary = pd.DataFrame({
        "Configuración básica": ablation_results.loc["Base + pos"][key_metrics],
        "Con contexto": context_results.loc["Con contexto"][key_metrics],
        "Modelo completo": context_results.loc["Completo (con gazetteers)"][key_metrics]
    })
    display(results_summary)
except:
    print("Ejecuta los experimentos anteriores para ver un resumen comparativo")

Resumen de experimentos:


,Configuración básica,Con contexto,Modelo completo
Balanced accuracy,0.611477,0.661735,0.679353
F1 Score,0.553191,0.602985,0.634731
Precision,0.572327,0.612121,0.646341
Recall,0.535294,0.594118,0.623529


In [99]:
def train_to_IO(train):
    '''
    Convert the train to IO format while preserving POS tags
    '''
    train_io = []
    for sent in train:
        sent_io = []
        for word, pos, tag in sent:
            new_tag = re.sub(r'\bB-', 'I-', tag)
            sent_io.append((word, pos, new_tag))
        train_io.append(sent_io)
    return train_io

esp_train_IO = train_to_IO(esp_train)
ned_train_IO = train_to_IO(ned_train)

# Imprimir un ejemplo de la conversión a IO
print("Ejemplo de oración original (español):")
print(esp_train[0])
print("\nEjemplo de oración en formato IO:")
print(esp_train_IO[0])

Ejemplo de oración original (español):
[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]

Ejemplo de oración en formato IO:
[('Melbourne', 'NP', 'I-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'I-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'I-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [100]:
def train_to_BIOE(train):
    '''
    Convert the train to BIOE format while preserving POS tags
    '''
    train_bioe = []
    for sent in train:
        sent_bioe = []
        for i in range(len(sent)):
            word, pos, tag = sent[i]
            if tag.startswith('I-') and (i == len(sent) - 1 or not sent[i + 1][2].startswith('I-')):
                tag = re.sub(r'\bI-', 'E-', tag)
            sent_bioe.append((word, pos, tag))
        train_bioe.append(sent_bioe)
    return train_bioe

esp_train_BIOE = train_to_BIOE(esp_train)  # Original BIO data
esp_train_BIOS = train_to_BIOE(esp_train)  # Original BIO data

# Imprimir un ejemplo de la conversión a BIOE
print("Ejemplo de oración original (español):")
print(esp_train[0])
print("\nEjemplo de oración en formato BIOE:")
print(esp_train_BIOE[0])

Ejemplo de oración original (español):
[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]

Ejemplo de oración en formato BIOE:
[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [101]:
def train_to_BIOS(train):
    '''
    Convert the train to BIOS format while preserving POS tags
    '''
    train_bios = []
    for sent in train:
        sent_bios = []
        for i in range(len(sent)):
            word, pos, tag = sent[i]
            if tag.startswith('B-') and (i == len(sent) - 1 or sent[i + 1][2] == 'O'):
                tag = re.sub(r'\bB-', 'S-', tag)
            sent_bios.append((word, pos, tag))
        train_bios.append(sent_bios)
    return train_bios

esp_train_BIOS = train_to_BIOS(esp_train)
ned_train_BIOS = train_to_BIOS(ned_train)

# Imprimir un ejemplo de la conversión a BIOS
print("Ejemplo de oración original (español):")
print(esp_train[0])
print("\nEjemplo de oración en formato BIOS:")
print(esp_train_BIOS[0])

Ejemplo de oración original (español):
[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]

Ejemplo de oración en formato BIOS:
[('Melbourne', 'NP', 'S-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'S-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'S-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [102]:
def train_to_BIOES(train):
    '''Convert BIO to BIOES directly'''
    train_bioes = []
    for sent in train:
        sent_bioes = []
        for i in range(len(sent)):
            word, pos, tag = sent[i]
            # Single-token entity (B-X followed by non-entity)
            if tag.startswith('B-') and (i == len(sent) - 1 or not sent[i + 1][2].startswith('I-')):
                tag = re.sub(r'\bB-', 'S-', tag)
            # End of multi-token entity
            elif tag.startswith('I-') and (i == len(sent) - 1 or not sent[i + 1][2].startswith('I-')):
                tag = re.sub(r'\bI-', 'E-', tag)
            sent_bioes.append((word, pos, tag))
        train_bioes.append(sent_bioes)
    return train_bioes

esp_train_BIOES = train_to_BIOES(esp_train)
ned_train_BIOES = train_to_BIOES(ned_train)

# Imprimir un ejemplo de la conversión a BIOES
print("Ejemplo de oración original (español):")
print(esp_train[0])
print("\nEjemplo de oración en formato BIOES:")
print(esp_train_BIOES[0])

Ejemplo de oración original (español):
[('Melbourne', 'NP', 'B-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'B-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'B-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]

Ejemplo de oración en formato BIOES:
[('Melbourne', 'NP', 'S-LOC'), ('(', 'Fpa', 'O'), ('Australia', 'NP', 'S-LOC'), (')', 'Fpt', 'O'), (',', 'Fc', 'O'), ('25', 'Z', 'O'), ('may', 'NC', 'O'), ('(', 'Fpa', 'O'), ('EFE', 'NC', 'S-ORG'), (')', 'Fpt', 'O'), ('.', 'Fp', 'O')]


In [103]:
def entity_finder_universal(sent_tags):
    """
    Finds entities in sentences based on tags in any encoding scheme (BIO, IO, BIOE, BIOS, BIOES).
    
    Args:
        sent_tags: List of sentences with tags
        
    Returns:
        List of entities found per sentence [(type, (start, end)), ...]
    """
    entities = []
    for sent in sent_tags:
        sent_entities = []  # List for current sentence
        entities.append(sent_entities)
        
        current_entity = None
        start_idx = None
        entity_type = None
        
        for i, tag in enumerate(sent):
            # Check if this is part of an entity (any entity tag except "O")
            if tag != "O" and ("-" in tag):
                prefix, entity = tag.split("-", 1)
                
                # Start of entity
                if prefix in ["B", "S"] or (prefix == "I" and current_entity is None):
                    # If we were already tracking an entity, add it to results
                    if current_entity is not None:
                        sent_entities.append((entity_type, (start_idx, i-1)))
                    
                    # Start tracking new entity
                    current_entity = entity
                    start_idx = i
                    entity_type = entity
                
                # Inside entity but type changed
                elif prefix == "I" and entity != current_entity:
                    if current_entity is not None:
                        sent_entities.append((entity_type, (start_idx, i-1)))
                    current_entity = entity
                    start_idx = i
                    entity_type = entity
                
                # End of entity
                elif prefix == "E":
                    if current_entity is not None:
                        sent_entities.append((entity_type, (start_idx, i)))
                        current_entity = None
                        start_idx = None
                        entity_type = None
                    
                # Single token entity
                if prefix == "S":
                    sent_entities.append((entity, (i, i)))
                    current_entity = None
                    start_idx = None
                    entity_type = None
                    
            # Not part of an entity
            else:  
                if current_entity is not None:  # End previous entity
                    sent_entities.append((entity_type, (start_idx, i-1)))
                    current_entity = None
                    start_idx = None
                    entity_type = None
        
        # If there's an entity at the end of the sentence
        if current_entity is not None:
            sent_entities.append((entity_type, (start_idx, len(sent)-1)))
    
    return entities

def evaluate_model_universal(y_true, y_pred, errors=False):
    """
    Evaluates model performance using multiple metrics with any encoding scheme.
    
    Args:
        y_true: True labels
        y_pred: Predicted labels
        errors: If True, also returns error list
        
    Returns:
        Dictionary with performance metrics
    """
    info = {'Balanced accuracy': 0.0, 'F1 Score': 0.0, 'Precision': 0.0, 'Recall': 0.0}
    
    # Calculate balanced accuracy
    def join_sent_tags(sent_tags):
        return [tag for sent in sent_tags for tag in sent]

    info['Balanced accuracy'] = balanced_accuracy_score(join_sent_tags(y_true), join_sent_tags(y_pred))

    # Find entities using the universal entity finder
    true_entities = entity_finder_universal(y_true)
    pred_entities = entity_finder_universal(y_pred)

    # Count real and correctly predicted entities by type
    counts = {'LOC': 0, 'MISC': 0, 'ORG': 0, 'PER': 0}
    correct_counts = {'LOC': 0, 'MISC': 0, 'ORG': 0, 'PER': 0}
    invented = 0

    for i, sent in enumerate(true_entities):
        sent_true = set(sent)
        sent_pred = set(pred_entities[i])
        
        # Count real entities by type
        for ent in sent:
            counts[ent[0]] += 1
            
        # Count correctly predicted entities
        for ent in sent_pred & sent_true:
            correct_counts[ent[0]] += 1
            
        # Count invented entities (false positives)
        invented += len(sent_pred - sent_true)

    # Calculate precision by entity type
    for ent_type in counts:
        total = counts[ent_type]
        correct = correct_counts[ent_type]
        info[f'{ent_type} correct'] = correct / total if total > 0 else 0.0

    # Calculate global metrics
    total_entities = sum(counts.values())
    true_positives = sum(correct_counts.values())
    false_positives = invented
    false_negatives = total_entities - true_positives
    
    # Precision, Recall and F1
    info['Precision'] = true_positives / (true_positives + false_positives) if (true_positives + false_positives) else 0
    info['Recall'] = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) else 0
    
    if info['Precision'] + info['Recall'] > 0:
        info['F1 Score'] = 2 * (info['Precision'] * info['Recall']) / (info['Precision'] + info['Recall'])
    else:
        info['F1 Score'] = 0

    # Return error list if requested
    if errors:
        error_list = []
        for i in range(len(true_entities)):
            error_list.extend([
                (i, ent) for ent in pred_entities[i] if ent not in true_entities[i]
            ])
        return info, error_list

    return info

In [104]:
import pandas as pd

# Definir los diferentes esquemas de codificación y sus datasets
train_sets_es = [esp_train_IO, esp_train_BIOE, esp_train_BIOS, esp_train_BIOES]
train_sets_nl = [ned_train_IO, ned_train_BIOE, ned_train_BIOS, ned_train_BIOES]

# Usar un subconjunto pequeño para acelerar la comparación
small_train_sets_es = [train_set[:1000] for train_set in train_sets_es]
small_train_sets_nl = [train_set[:1000] for train_set in train_sets_nl]

# Evaluar el modelo para cada esquema de codificación en español
results_es = []
processor_es = NERDataProcessor("spanish")
dev_es = list(esp_dev)


X_dev_es = processor_es.convert_to_features(dev_es)
y_dev_es = processor_es.get_labels(dev_es)

for train_set in small_train_sets_es:
    X_train_es = processor_es.convert_to_features(train_set)
    y_train_es = processor_es.get_labels(train_set)
    feature_gen = CRFFeatureGenerator({
        "word_form": True,
        "pos": True,
        "morphology": True,
        "prefix_suffix": True,
        "length": True,
        "position": True,
        "context": True,
        "gazetteers": True
    })
    model = CRFModel(feature_gen)
    model.train(X_train_es, y_train_es)
    y_pred_es = model.predict(X_dev_es)

    idx = small_train_sets_es.index(train_set)
    if idx == 1:  # BIOE
        dev_es_scheme = train_to_BIOE(list(esp_dev))
    elif idx == 2:  # BIOS
        dev_es_scheme = train_to_BIOS(list(esp_dev))
    elif idx == 3:  # BIOES
        dev_es_scheme = train_to_BIOES(list(esp_dev))
    else:  # IO
        dev_es_scheme = list(esp_dev)
    X_dev_es = processor_es.convert_to_features(dev_es_scheme)
    y_dev_es = processor_es.get_labels(dev_es_scheme)
    # Actualizar las características y etiquetas de dev
    eval_result = evaluate_model_universal(y_dev_es, y_pred_es)
    results_es.append(eval_result)

df_results_es = pd.DataFrame(results_es, index=["IO", "BIOE", "BIOS", "BIOES"])
display(df_results_es)

# Evaluar el modelo para cada esquema de codificación en holandés
results_nl = []
processor_nl = NERDataProcessor("dutch")
dev_nl = list(ned_dev)
X_dev_nl = processor_nl.convert_to_features(dev_nl)
y_dev_nl = processor_nl.get_labels(dev_nl)

for train_set in small_train_sets_nl:
    X_train_nl = processor_nl.convert_to_features(train_set)
    y_train_nl = processor_nl.get_labels(train_set)
    feature_gen = CRFFeatureGenerator({
        "word_form": True,
        "pos": True,
        "morphology": True,
        "prefix_suffix": True,
        "length": True,
        "position": True,
        "context": True,
        "gazetteers": True
    })
    model = CRFModel(feature_gen)
    model.train(X_train_nl, y_train_nl)
    y_pred_nl = model.predict(X_dev_nl)

    # Convert dev data to match training data scheme for Dutch
    idx = small_train_sets_nl.index(train_set)
    if idx == 1:  # BIOE
        dev_nl_scheme = train_to_BIOE(list(ned_dev))
    elif idx == 2:  # BIOS
        dev_nl_scheme = train_to_BIOS(list(ned_dev))
    elif idx == 3:  # BIOES
        dev_nl_scheme = train_to_BIOES(list(ned_dev))
    else:  # IO
        dev_nl_scheme = list(ned_dev)

    X_dev_nl = processor_nl.convert_to_features(dev_nl_scheme)
    y_dev_nl = processor_nl.get_labels(dev_nl_scheme)

    eval_result = evaluate_model_universal(y_dev_nl, y_pred_nl)
    results_nl.append(eval_result)

df_results_nl = pd.DataFrame(results_nl, index=["IO", "BIOE", "BIOS", "BIOES"])
display(df_results_nl)


Modelo entrenado y guardado como 'model.crf'
Modelo entrenado y guardado como 'model.crf'
Modelo entrenado y guardado como 'model.crf'
Modelo entrenado y guardado como 'model.crf'


,Balanced accuracy,F1 Score,Precision,Recall,LOC correct,MISC correct,ORG correct,PER correct
IO,0.367282,0.609563,0.638721,0.582950,0.658883,0.204494,0.572941,0.673486
BIOE,0.547884,0.628394,0.652324,0.606158,0.680203,0.202247,0.624706,0.667758
BIOS,0.561564,0.620277,0.646560,0.596048,0.680203,0.202247,0.617647,0.641571
BIOES,0.526705,0.622408,0.646607,0.599954,0.681218,0.182022,0.633529,0.639935


Modelo entrenado y guardado como 'model.crf'
Modelo entrenado y guardado como 'model.crf'
Modelo entrenado y guardado como 'model.crf'
Modelo entrenado y guardado como 'model.crf'


,Balanced accuracy,F1 Score,Precision,Recall,LOC correct,MISC correct,ORG correct,PER correct
IO,0.314452,0.585576,0.633052,0.544725,0.682672,0.449198,0.380466,0.712660
BIOE,0.306811,0.294614,0.617827,0.193425,0.010438,0.050802,0.166181,0.496444
BIOS,0.488673,0.581514,0.623632,0.544725,0.693111,0.461230,0.332362,0.739687
BIOES,0.417516,0.581087,0.613424,0.551988,0.697286,0.482620,0.332362,0.741110
